---

# <span style="color:orange; font-weight:bold;">Regulatory AI Search Engine with Llama</span> <br>
### <span style="color:orange; font-weight:bold;">A Local Retrieval-Augmented Generation (RAG) </span> <br> <br>
#### <span style="color:orange; font-weight:bold;">Script to create and maintain the vectorDB </span> <br> <br>
*created with ❤️ by Daniel Jacobowitz & Prasoon Singh*

---

In [2]:
#!pip install langchain-community==0.2.4 langchain==0.2.3 faiss-cpu==1.8.0 unstructured==0.14.5 unstructured[pdf]==0.14.5 transformers==4.41.2 sentence-transformers==3.0.1

import pandas as pd

import os # --> interacts with the operating system, like creating files and directories, management of files and directories, input, output, environment variables etc.

#import pandas as pd # --> Analyses data and allows to manipulate data as dataframes

from langchain_community.llms import Ollama  # --> LLM classes provide access to the large language model (in this case locally to Ollama) APIs and services.

from langchain.document_loaders import UnstructuredFileLoader # --> Loads data from a source as Documents, which are a piece of text and associated metadata.
                                                              # --> In this case, the file loader uses the unstructured partition function and will automatically detect the file type.

from langchain.document_loaders.pdf import PyPDFDirectoryLoader # --> Loads data from a source as Documents, which are a piece of text and associated metadata.
                                                                # --> In this case, the file loader loads a directory with PDF files using pypdf and chunks at character level. The loader also stores page numbers in metadata.

from langchain_community.document_loaders import PyPDFLoader

from langchain.embeddings import HuggingFaceEmbeddings #--> Convert the text of the PDF files using Embeddings models into numerical vectors
#from langchain_community.embeddings.ollama import OllamaEmbeddings #--> Convert the text of the PDF files using Embeddings models into numerical vectors
#from langchain_ollama import OllamaEmbeddings #--> Convert the text of the PDF files using Embeddings models into numerical vectors

from langchain_community.vectorstores import FAISS # --> Creates a local vector store to store the created embeddings of the PDF files. LangChain has a number of built-in document transformers that make it easy to split, combine, filter, and otherwise manipulate documents.


from langchain.text_splitter import CharacterTextSplitter # --> Splits a long document (in our case the PDF files) into smaller chunks that can fit into the model`s context window. LangChain has a number of built-in document transformers that make it easy to split, combine, filter, and otherwise manipulate documents.
from langchain.text_splitter import RecursiveCharacterTextSplitter # --> Splits a long document (in our case the PDF files) into smaller chunks that can fit into the model`s context window. LangChain has a number of built-in document transformers that make it easy to split, combine, filter, and otherwise manipulate documents.
from langchain_experimental.text_splitter import SemanticChunker #--> Splits a long document (in our case the PDF files) into smaller chunks that can fit into the model`s context window. LangChain has a number of built-in document transformers that make it easy to split, combine, filter, and otherwise manipulate documents.

from langchain.chains import RetrievalQA # --> Chains refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. This chain first does a retrieval step to fetch relevant documents, then passes those documents into an LLM to generate a response.

from langchain_core.prompts import ChatPromptTemplate # --> A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.
#from langchain.memory import ConversationBufferMemory # --> Creates a conversational memory that allows a LLM to remember previous interactions/ chats with the user. It helps to manage and store conversation history in a structured way and to maintain the context of a conversation over multiple interactions.

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # --> LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. In this case, it is a callback for streaming.
                                                                                # --> Streaming: process of incrementally receiving and processing data generated by a Large Language Model (LLM) as it is produced, rather than waiting for the entire response to be generated before displaying it.
from langchain.callbacks.manager import CallbackManager # --> LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. In this case, it is to manage callbacks.

import streamlit as st # --> Streamlit is a free and open-source framework to rapidly build and share beautiful machine learning and data science web apps.

import time #This module provides various time-related functions. 

if not os.path.exists('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/pdfFiles'): # Check if the folder does not exist
    os.makedirs('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/pdfFiles') # create a folder in the specified path if it does not exist already

if not os.path.exists('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB'): # Check if the folder does not exist
    os.makedirs('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB') # create a folder in the specified path if it does not exist already

# to initialize a new vector database, change to 1
switch = 0

---

In [3]:
# 1. Upload relevant PDF files
DATA_PATH = "C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/pdfFiles" # Directory where the relevant PDFs are stored

if switch == 1:
    def load_documents(DATA_PATH): # defines a function named 'load_documents' that takes one parameter, 'DATA_PATH'.
        document_loader = PyPDFDirectoryLoader(DATA_PATH) # creates the class 'PyPDFDirectoryLoader' designed to load PDF documents from a specified directory.
        return document_loader.load() # the load method on the document_loader instance and returns the result. The load method is expected to read and load the PDF documents from the directory specified by DATA_PATH. The load method loads data into Document objects.

    documents = load_documents(DATA_PATH) # the variable 'documents' is a object containing for every list elements a page of the PDF files.
    
    # --> Plausibility check
    print("\033[92mThe upload of the PDF files was successful!\033[0m")
    print('')
    print('CRR:')
    print (documents[0])
    print('')
    print('Banking Act:')
    print(documents[337])
    print('')
    print('MaRisk:')
    print(documents[600])
    print('')

In [4]:
# 2. Chunking the documents

if switch == 1:
    #text_splitter = SemanticChunker(embeddings=HuggingFaceEmbeddings(), breakpoint_threshold_type="percentile") # Apply the chunking based on cosine similarity. There a many breakpoint threshold types.
    #text_chunks = text_splitter.split_documents(documents) # #  splits the documents into chunks based on the similarity
    
    text_splitter = RecursiveCharacterTextSplitter( # function to split documents into chunks
                chunk_size=1500, # sets the maximum size of each chunk to 1500 characters.
                chunk_overlap=200, #  sets the overlap between consecutive chunks to 200 characters. Overlapping helps to ensure that context is preserved across chunks.
                length_function=len # specifies that the length of the chunks will be measured using the len function, which counts the number of characters.
            )

    text_chunks = text_splitter.split_documents(documents) #  splits the documents into chunks based on the specified chunk_size and chunk_overlap.

In [5]:
# 3. Convert the splitted text into Vector Embeddings incl. indexation and store the created vector data base locally

embeddings = HuggingFaceEmbeddings() # The used embeddings model is from HuggingFace
 #embeddings = OllamaEmbeddings(
    #        model = "llama3.2:3b",
    #  base_url = 'http://localhost:11434'
    #    )

if switch == 1:
   # Create and save the vector store
    knowledge_base = FAISS.from_documents(text_chunks, embeddings) # The created chunks are converted to numerical vectors and stored in a FAISS vector database.
    knowledge_base.save_local('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB') # The created FAISS vector database is stored locally in the given directory.

    print("\033[92mThe Embedding was successful!\033[0m")


C:\Users\49171\AppData\Local\Temp\ipykernel_25452\1986862643.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings() # The used embeddings model is from HuggingFace
C:\Users\49171\AppData\Local\Temp\ipykernel_25452\1986862643.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # The used embeddings model is from HuggingFace
C:\Users\49171\PycharmProjects\pythonProject3\.venv\lib\site-packages\sentence_transformers\cross_encoder\Cross

---

In [6]:
# this cell is to manage an existing the VectorDB

db = FAISS.load_local( # This method converts the stored vector database into a retriever object. A retriever is typically used to search and retrieve relevant documents or information. In other words, it will be the base for the search as the retrieved information will then later be passed to the LLM.
    'C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB', # Directory of the locally stored vector database
    embeddings, # Parameter to declare, that the embeddings should be loaded.
    allow_dangerous_deserialization=True)


text_splitter = RecursiveCharacterTextSplitter( # function to split documents into chunks
                chunk_size=1500, # sets the maximum size of each chunk to 1500 characters.
                chunk_overlap=200, #  sets the overlap between consecutive chunks to 200 characters. Overlapping helps to ensure that context is preserved across chunks.
                length_function=len # specifies that the length of the chunks will be measured using the len function, which counts the number of characters.
            )

embeddings = HuggingFaceEmbeddings() # The used embeddings model is from HuggingFace
 #embeddings = OllamaEmbeddings(
    #        model = "llama3.2:3b",
    #  base_url = 'http://localhost:11434'
    #    )

# function to display documents in vectorDB
def show_vstore(store):
    vector_df = store_to_df(store)
    print(vector_df)

# function to convert a vector store into df to convenient access
def store_to_df(store):
    v_dict = store.docstore._dict
    data_rows = []
    for k in v_dict.keys():
        source_path = v_dict[k].metadata["source"].split("/")[-1]
        source_filename = os.path.basename(source_path)
        doc_name = source_filename
        
        page_number = v_dict[k].metadata["page"]+1
        
        content = v_dict[k].page_content
        
        data_rows.append({"chunk_id":k, "document":doc_name,"page":page_number,"content":content})
    vector_df = pd.DataFrame(data_rows)
    return vector_df



C:\Users\49171\AppData\Local\Temp\ipykernel_25452\2681690023.py:15: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings() # The used embeddings model is from HuggingFace


In [7]:
# Test to see the vectorDB.
show_vstore(db)

,chunk_id,document,page,content
0,d830feca-520c-496e-ab58-dcd58b70504f,CRR.pdf,1,I \n(Legislative acts) \nREGULATIONS \nREGUL...
1,af3b5a0c-494f-48ab-9a5c-843854c08145,CRR.pdf,1,stronger liquidity buffers. In response to the...
2,a688ad37-c9d3-43d0-b3cd-63ba9262a6ad,CRR.pdf,1,national regulatory measures considered to be ...
3,3c44f466-4629-4acf-8952-93aad77b5eb2,CRR.pdf,2,the Council of 14 June 2006 on the capital ade...
4,4bc26976-f6df-49f3-8384-7a4c98f286b7,CRR.pdf,2,concerning the access to the activity of insti...
...,...,...,...,...
4257,cbc10643-ad89-4170-9b93-c258d8fc5ba1,MaRisk.pdf,117,the proposals for action with the responsible ...
4258,3cc3400d-138c-4013-8e91-4a3103b2b116,MaRisk.pdf,118,Bundesanstalt für Finanzdienstleistungsaufsich...
4259,aebe2119-3ea9-4e44-9624-2e8edb1546e3,MaRisk.pdf,118,"office, and\ni)in the case of institutions wit..."
4260,5ca7e410-6d7f-472c-bc3c-85a3849069f4,MaRisk.pdf,119,Bundesanstalt für Finanzdienstleistungsaufsich...


In [9]:
# Check, if new files has been added to the directory and has to be added to the existing vectorDB.

vector_df = store_to_df(db)
file_names = os.listdir(DATA_PATH)

missing_files = [file for file in file_names if file not in vector_df["document"].values]

if not missing_files:
    print("\033[92mNo documents are added to the directory. No adjustment of the vector database is required.\033[0m")
else:
    print("\033[92mNew documents has been identified.\033[0m")
    time.sleep(1)
    print("\033[91mThe following documents are new:\033[0m", missing_files)
    print("\033[91mThe new documents will be added now to the vector database.\033[0m")
    print("")
    time.sleep(1)
    print("Please wait...")
    print("")
   
    missing_files_full_path = [os.path.join(DATA_PATH, file) for file in missing_files]
    merged_documents = []
    for missings in missing_files_full_path:
        document_loader = PyPDFLoader(missings)
        document_new = document_loader.load()
        merged_documents += document_new
    print("\033[92mThe new files has been added successfully to the vector database.\033[0m")
    print("")
    time.sleep(1)
    print("Now the new files will be chunked.")
    text_chunks_new = text_splitter.split_documents(merged_documents)
    extension = FAISS.from_documents(text_chunks_new, embeddings)
    print("")
    print("\033[92mThe text chunking was successful! Now the new chunks will be added to the existing vector database.\033[0m")
    db.merge_from(extension)
    db.save_local('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB') # The created FAISS vector database is stored locally in the given directory.
    print("")
    print("\033[92mThe merging was successful! The local vector database is now updated. See you next time!\033[0m")

# Test
show_vstore(db)

Missing documents has been identified.
he following documents are missing: ['MaRisk_v2.pdf']
The missing documents will be added now to the vector database.

Please wait...

The missing files has been added successfully to the vector database.

Now the missing files will be chunked.

The text chunking was successful! Now the new chunks will be added to the existing vector database.

The merging was successful! The local vector database is now updated. See you next time!


,chunk_id,document,page,content
0,d830feca-520c-496e-ab58-dcd58b70504f,CRR.pdf,1,I \n(Legislative acts) \nREGULATIONS \nREGUL...
1,af3b5a0c-494f-48ab-9a5c-843854c08145,CRR.pdf,1,stronger liquidity buffers. In response to the...
2,a688ad37-c9d3-43d0-b3cd-63ba9262a6ad,CRR.pdf,1,national regulatory measures considered to be ...
3,3c44f466-4629-4acf-8952-93aad77b5eb2,CRR.pdf,2,the Council of 14 June 2006 on the capital ade...
4,4bc26976-f6df-49f3-8384-7a4c98f286b7,CRR.pdf,2,concerning the access to the activity of insti...
...,...,...,...,...
4535,84e73b2c-5c3e-4388-9dbf-ee4dfe68449c,MaRisk_v2.pdf,117,the proposals for action with the responsible ...
4536,98ce08a3-50c7-47e9-bd96-6b28351baf28,MaRisk_v2.pdf,118,Bundesanstalt für Finanzdienstleistungsaufsich...
4537,19701d48-d0bd-4a60-8513-07d9449a2b40,MaRisk_v2.pdf,118,"office, and\ni)in the case of institutions wit..."
4538,6e2b0b8d-599a-4d3f-9077-0c453d99b02a,MaRisk_v2.pdf,119,Bundesanstalt für Finanzdienstleistungsaufsich...


In [11]:
# Check, if files has been deleted from the initial directory and has to be deleted from the existing vectorDB.

vector_df = store_to_df(db)
file_names = os.listdir(DATA_PATH)

deleted_documents = [file for file in vector_df["document"].values if file not in file_names]

if not deleted_documents:
    print("\033[92mNo Documents has been deleted from the directory. The vector database has not to be updated.\033[0m")
else:
    deleted_documents = list(set(deleted_documents))
    print("\033[91mDeleted documents in the directory has been identified.\033[0m")
    time.sleep(1)
    print("\033[91mThe following documents has been deleted:\033[0m", deleted_documents)
    print("")
    time.sleep(1)
    print("\033[92mThe deleted documents will be deleted now from the vector database.\033[0m")
    print("")
    time.sleep(1)
    print("Please wait...")
    print("")
    
    vector_df = store_to_df(db)
    
    for deleted_doc in deleted_documents:
        chunk_list_delete = vector_df.loc[vector_df["document"] == deleted_doc]["chunk_id"].tolist()
        db.delete(chunk_list_delete)
    print("")
    print("\033[92mThe deletion of the relevant documents from the vector database was successful!.\033[0m")
    db.save_local('C:/Users/49171/OneDrive/Desktop/Programieren/Python/PDF_test/Final/vectorDB')
    print("")
    print("\033[92mThe update is completed. See you next time!\033[0m")
# Test
show_vstore(db)   
    
    
    

Deleted documents in the directory has been identified.
The following documents has been deleted: ['MaRisk_v2.pdf']

The deleted documents will be deleted now from the vector database.

Please wait...


The deletion of the relevant documents from the vector database was successful!.

The update is completed. See you next time!


,chunk_id,document,page,content
0,d830feca-520c-496e-ab58-dcd58b70504f,CRR.pdf,1,I \n(Legislative acts) \nREGULATIONS \nREGUL...
1,af3b5a0c-494f-48ab-9a5c-843854c08145,CRR.pdf,1,stronger liquidity buffers. In response to the...
2,a688ad37-c9d3-43d0-b3cd-63ba9262a6ad,CRR.pdf,1,national regulatory measures considered to be ...
3,3c44f466-4629-4acf-8952-93aad77b5eb2,CRR.pdf,2,the Council of 14 June 2006 on the capital ade...
4,4bc26976-f6df-49f3-8384-7a4c98f286b7,CRR.pdf,2,concerning the access to the activity of insti...
...,...,...,...,...
4257,cbc10643-ad89-4170-9b93-c258d8fc5ba1,MaRisk.pdf,117,the proposals for action with the responsible ...
4258,3cc3400d-138c-4013-8e91-4a3103b2b116,MaRisk.pdf,118,Bundesanstalt für Finanzdienstleistungsaufsich...
4259,aebe2119-3ea9-4e44-9624-2e8edb1546e3,MaRisk.pdf,118,"office, and\ni)in the case of institutions wit..."
4260,5ca7e410-6d7f-472c-bc3c-85a3849069f4,MaRisk.pdf,119,Bundesanstalt für Finanzdienstleistungsaufsich...
